In [ ]:
import os
import polars as pl
import tensorflow as tf

# Import CMI evaluation system
import kaggle_evaluation.cmi_inference_server

# Import our inference module
from model_inference import predict_gesture

print("Libraries imported successfully")
print(f"TensorFlow version: {tf.__version__}")


In [ ]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    Main prediction function for Kaggle evaluation system
    
    Args:
        sequence: Polars DataFrame containing sensor data sequence
        demographics: Polars DataFrame containing participant demographics
    
    Returns:
        str: Predicted gesture name
    """
    # Convert Polars DataFrame to Pandas DataFrame
    sequence_df = sequence.to_pandas()
    
    # Predict gesture using our trained LSTM model
    gesture = predict_gesture(sequence_df)
    
    return gesture


print("Kaggle evaluation prediction function defined")


In [ ]:
# Initialize the inference server
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

# Start the server
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # Competition evaluation mode
    inference_server.serve()
else:
    # Local testing mode (fallback)
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )
